In [ ]:
from __future__ import print_function
import os.path
import ROOT

# ROOT.gSystem.AddDynamicPath("/work/dune-docker-home/ups/duneanaobj/v01_01_00/slf7.x86_64.e15.gv1.debug/lib")
# ROOT.gSystem.Load("libduneanaobj_StandardRecord")
# ROOT.gSystem.Load("libduneanaobj_StandardRecord_dict")
prefix = "/home/jeremy/data" if os.path.isdir("/home/jeremy") else ""
oldcaf = ROOT.TFile(prefix + "/dune/data/users/jwolcott/nd/nd-lar-reco/caf/FHC.1000001.CAF.root")
newcaf = ROOT.TFile(prefix + "/dune/data/users/jwolcott/scratch/FHC.1000001.CAF.root")

assert not any(f.IsZombie() for f in (oldcaf, newcaf))
print("files loaded ok")

In [ ]:
t = newcaf.Get("globalTree")
t.GetEntry(0)
g = getattr(t, "global")
print(g.wgts.params[0].name)

wgt_branches = [p.name for p in g.wgts.params]
# for p in g.wgts.params:
#      print(p.name)
# for syst_param_header in b.GetValuePointer():
#     print(syst_param_header.name)
# for w in b.GetValue():
#     print(w.name)
print(wgt_branches)

In [ ]:
import pprint

trees = {
    "old": oldcaf.Get("caf"),
    "new": newcaf.Get("cafTree"),
}
varlist = [b.GetName() for b in trees["old"].GetListOfBranches()]
print(len(varlist), "total branches from old tree:", varlist)

matched_branches = set([s for s in varlist if trees["new"].GetBranch("rec.%s" % s) or trees["new"].GetBranch(s)])
unmatched_branches = set(varlist) - matched_branches

old_wgt_branches = set()
recovered_wgt_branches = 0
for br in set(unmatched_branches):
    br_root = None
    if br.startswith("wgt_"):
        br_root = br[4:]
    elif br.endswith("_cvwgt"):
        br_root = br[:-6]
    elif br.endswith("_nshifts"):
        br_root = br[:-8]
    else:
        continue
#    print("looking at weight branch root:", br_root)

    if br_root in wgt_branches:
#        print('matched:', br_root)
        old_wgt_branches.add(br_root)
        unmatched_branches.remove(br)
        recovered_wgt_branches += 1

print(len(matched_branches) + recovered_wgt_branches, "matched branches:", pprint.pformat(matched_branches | old_wgt_branches))
if len(unmatched_branches) > 0:
    print(len(unmatched_branches), "unmatched branches:", pprint.pformat(sorted(unmatched_branches)))

In [ ]:
print(trees["new"].GetBranch("Q2"))

In [ ]:
def check_branch(old_brname, new_brname, trees, canvases):
    trees["old"].Draw("%s>>hold_%s" % (old_brname, old_brname), "", "goff")
    hold = getattr(ROOT, "hold_%s" % old_brname)

    hnew = ROOT.TH1F(hold)
    hnew.SetName("hnew_%s" % new_brname)
    trees["new"].Draw("rec.%s>>hnew_%s" % (new_brname, new_brname), "", "goff")

    diff = ROOT.TH1F(hnew)
    diff.SetName("diff_%s" % old_brname)
    diff.Add(hold, -1)

    integral = diff.Integral()

    if integral != 0:
        print(old_brname, "integral nonzero:", integral)
        c = ROOT.TCanvas()

        hold.DrawCopy()

        hnew.SetLineColor(ROOT.kRed)
        hnew.DrawCopy("same")
        c.Draw()

        canvases.append(c)
    else:
         print(old_brname, "is ok")

    return canvases

canvases = []
for var in matched_branches:
    check_branch(var, var, trees, canvases)

for br_root in old_wgt_branches:
    for old_brname_template, new_brname_template in \
		    (("wgt_%s", "rec.xsSyst_wgt[%d]"), ("%s_cvwgt", "rec.cvwgt[%d]")):
        check_branch(old_brname_template % br_root, new_brname_template % wgt_branches.index(br_root),
                     trees, canvases)